<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Analyzing%20Structured%20data/Natural_Language_Query_Interface_RDBMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import openai
import pandas as pd
import time
import re
from IPython.display import Markdown, display
from google.colab import files
import sqlite3

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from openai import OpenAI

from google.colab import userdata

client = OpenAI(api_key= userdata.get('secretName'))

In [ ]:
def get_structure(data_path):
    """ Extract structure from SQLite database.

    Args:
        data_path: path to SQLite data file.

    Returns:
        text description of database structure.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute("select sql from sqlite_master where type ='table';")
        table_rows = cursor.fetchall()
        table_ddls = [r[0] for r in table_rows]
        return '\n'.join(table_ddls)

In [ ]:
def create_prompt(description, question):
    """ Generate prompt to translate question into SQL query.

    Args:
        description: text description of database structure.
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += [description]
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

In [ ]:
def call_llm(prompt):
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {'role':'user', 'content':prompt}
                    ]
                )
            return response.choices[0].message.content
        except:
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query OpenAI model!')

In [ ]:
def process_query(data_path, query):
    """ Processes SQL query and returns result.

    Args:
        data_path: path to SQLite data file.
        query: process this query on database.

    Returns:
        query result.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        table_rows = cursor.fetchall()
        table_strings = [str(r) for r in table_rows]
        return '\n'.join(table_strings)

In [ ]:
db_path = '/content/drive/MyDrive/Colab Notebooks/DataScience+GPT/Data/structured_data/games.db'

data_structure = get_structure(db_path)
print(data_structure)

while True:

  user_input = input('Enter question:')
  if user_input == 'quit':
    break

  prompt = create_prompt(data_structure, user_input)
  print(prompt)
  answer = call_llm(prompt)
  query = re.findall('```sql(.*)```', answer, re.DOTALL)[0]
  print(f'SQL: {query}')

  try:
    result = process_query(db_path, query)
    print(f'Result: {result}')
  except:
    print('Error processing query! Try to reformulate.')

CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Enter question:Break down game sales in Europe by the platform!
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question into SQL query:
Break down game sales in Europe by the platform!
SQL: 
SELECT platform, SUM(eusales) AS total_europe_sales
FROM games
GROUP BY platform;

Result: ('2600', 5.46999999999998)
('3DO', 0)
('3DS', 58.52000000000003)
('DC', 1.6900000000000002)
('DS', 194.64999999999938)
('GB', 47.82)
('GBA', 75.25000000000061)
('GC', 38.71000000000004)
('GEN', 5.5200000000000005)
('GG', 0)
('N64', 41.060000000000045)
('NES', 21.150000000000006)
('NG', 0)
('PC', 139.68000000000015)
('PCFX', 0)
('PS', 213.60000000000065